In [60]:
import sys
import cv2
import numpy as np

src = cv2.imread('namecard1.jpg')

if src is None:
    print('image load failed')
    sys.exit()

# src = cv2.resize(src, (640, 480))
#src = cv2.resize(src, (0, 0), fx=0.5, fy=0.5)

# 흑백흑백
src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

# 이진화 => 임계값 함수 => otsu를 쓰니 알아서 하라고 0을 줌.
_, src_bin = cv2.threshold(src_gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

In [61]:
contours, _ = cv2.findContours(src_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

print(len(contours))

for pts in contours:
    if cv2.contourArea(pts) < 1000:
        continue
        
    approx = cv2.approxPolyDP(pts, cv2.arcLength(pts, True)*0.02, True)
    
    if len(approx) != 4:
        continue
    
    w, h = 900, 500
    srcQuad = np.array([[approx[0, 0, :]], [approx[1, 0, :]], \
                        [approx[2, 0, :]], [approx[3, 0, :]]]).astype(np.float32)
    dstQuad = np.array([[0, 0], [0, h], [w, h], [w, 0]]).astype(np.float32)
    
    pers = cv2.getPerspectiveTransform(srcQuad, dstQuad)
    dst = cv2.warpPerspective(src, pers, (w, h)) 
    
    cv2.polylines(src, pts, True, (0, 0, 255))

111


In [ ]:
# cv2.imshow('src',src)
# cv2.imshow('src_gray',src_gray)
# cv2.imshow('src_bin',src_bin)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()